In [1]:
import re
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

# Data Cleaning and Text Preprocessing

In [2]:
train = pd.read_csv('data/labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)

In [3]:
train.shape

(25000, 3)

In [4]:
train.columns

Index(['id', 'sentiment', 'review'], dtype='object')

In [5]:
train.head(2)

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."


In [6]:
# There are many other things we could do to the data - For example, Porter Stemming and Lemmatizing 
# (both available in NLTK) would allow us to treat "messages", "message", and "messaging" as the same word, 
# which could certainly be useful. However, for simplicity, the tutorial will stop here.

def review_to_words(raw_review):
    """Converts a raw review into a string of words.
    
    Returns: str
    """
    # Remove HTML
    review_text = BeautifulSoup(raw_review, 'lxml').get_text()
    # Remove non-letters
    letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
    # covert to lower case and get split indiv. words
    words = letters_only.lower().split()
    # searching a set is faster than a list
    stops = set(stopwords.words('english'))
    # remove stopwords
    meaningful_words = [w for w in words if w not in stops]
    
    # join words into a string
    return ' '.join(meaningful_words)

In [7]:
clean_review = review_to_words( train['review'][0] )
print(clean_review)

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [8]:
clean_train = train.copy()
clean_train['review'] = clean_train['review'].apply(review_to_words)
clean_train.head(2)

,id,sentiment,review
0,"""5814_8""",1,stuff going moment mj started listening music ...
1,"""2381_9""",1,classic war worlds timothy hines entertaining ...


# Creating Features from a Bag of Words (Using scikit-learn)

In [9]:
# bag of words tool
vectorizer = CountVectorizer(analyzer='word',
                            tokenizer=None,
                            preprocessor=None,
                            stop_words=None,
                            max_features=5000)
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_train['review'])

In [10]:
# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
train_data_features.shape

(25000, 5000)

In [11]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

['abandoned', 'abc', 'abilities', 'ability', 'able', 'abraham', 'absence', 'absent', 'absolute', 'absolutely', 'absurd', 'abuse', 'abusive', 'abysmal', 'academy', 'accent', 'accents', 'accept', 'acceptable', 'accepted', 'access', 'accident', 'accidentally', 'accompanied', 'accomplished', 'according', 'account', 'accuracy', 'accurate', 'accused', 'achieve', 'achieved', 'achievement', 'acid', 'across', 'act', 'acted', 'acting', 'action', 'actions', 'activities', 'actor', 'actors', 'actress', 'actresses', 'acts', 'actual', 'actually', 'ad', 'adam', 'adams', 'adaptation', 'adaptations', 'adapted', 'add', 'added', 'adding', 'addition', 'adds', 'adequate', 'admire', 'admit', 'admittedly', 'adorable', 'adult', 'adults', 'advance', 'advanced', 'advantage', 'adventure', 'adventures', 'advertising', 'advice', 'advise', 'affair', 'affect', 'affected', 'afford', 'aforementioned', 'afraid', 'africa', 'african', 'afternoon', 'afterwards', 'age', 'aged', 'agent', 'agents', 'ages', 'aging', 'ago', 'ag

In [12]:
# Sum up the counts of each vocabulary word
dist = np.sum(train_data_features, axis=0)

# For each, print the vocabulary word and the number of times it 
# appears in the training set
for tag, count in zip(vocab, dist):
    print('{0}\t{1}'.format(count, tag))

187	abandoned
125	abc
108	abilities
454	ability
1259	able
85	abraham
116	absence
83	absent
352	absolute
1485	absolutely
306	absurd
192	abuse
91	abusive
98	abysmal
297	academy
485	accent
203	accents
300	accept
130	acceptable
144	accepted
92	access
318	accident
200	accidentally
88	accompanied
124	accomplished
296	according
186	account
81	accuracy
284	accurate
123	accused
179	achieve
139	achieved
124	achievement
90	acid
971	across
1251	act
658	acted
6490	acting
3354	action
311	actions
83	activities
2389	actor
4486	actors
1219	actress
369	actresses
394	acts
793	actual
4237	actually
148	ad
302	adam
98	adams
453	adaptation
80	adaptations
154	adapted
810	add
439	added
166	adding
347	addition
337	adds
113	adequate
124	admire
621	admit
134	admittedly
101	adorable
510	adult
376	adults
100	advance
90	advanced
153	advantage
510	adventure
204	adventures
91	advertising
259	advice
90	advise
346	affair
93	affect
113	affected
104	afford
126	aforementioned
343	afraid
212	africa
255	african
187	afternoon

# Random Forest

In [13]:
# Initialize a Random Forest classifier with 100 trees
forest = RandomForestClassifier(n_estimators = 100) 

# Fit the forest to the training set, using the bag of words as 
# features and the sentiment labels as the response variable
#
# This may take a few minutes to run
forest = forest.fit( train_data_features, train["sentiment"] )

# Submission

In [14]:
# Read the test data
test = pd.read_csv('data/testData.tsv', header=0, delimiter="\t", quoting=3 )

# Verify that there are 25,000 rows and 2 columns
print(test.shape)

# Create an empty list and append the clean reviews one by one
num_reviews = len(test["review"])

print("Cleaning and parsing the test set movie reviews...\n")
clean_test = test.copy()
clean_test['review'] = clean_test['review'].apply(review_to_words)

# Get a bag of words for the test set, and convert to a numpy array
test_data_features = vectorizer.transform(clean_test['review'])
test_data_features = test_data_features.toarray()

# Use the random forest to make sentiment label predictions
result = forest.predict(test_data_features)

# Copy the results to a pandas dataframe with an "id" column and
# a "sentiment" column
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )

(25000, 2)
Cleaning and parsing the test set movie reviews...



* [Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words)
* [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

Congratulations, you are ready to make your first submission! Try different things and see how 
your results change. You can clean the reviews differently, choose a different number of vocabulary 
words for the Bag of Words representation, try Porter Stemming, a different classifier, or any number 
of other things. To try out your NLP chops on a different data set, you can also head over to our 
[Rotten Tomatoes](https://www.kaggle.com/c/sentiment-analysis-on-movie-reviews) competition. Or, if you're ready for something completely different, move along to the 
Deep Learning and Word Vector pages.